## Twitter interface for the microtales generator

The Twitter account https://twitter.com/talesmachine is the interface with the microtales generator. Any user can mention @talesmachine and include a seed, and the generator will tweet the generated microtale

In [40]:
from __future__ import print_function
import numpy as np
from keras.models import model_from_yaml

local_path = "/home/ubuntu/data/"

In [41]:
import tweepy

#Twitter API credentials
consumer_key = ""
consumer_secret = ""
access_key = ""
access_secret = ""

In [42]:
#authorize twitter, initialize tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [43]:
import tinys3

#AWS credentials, to download the last model from S3
aws_access = {}
aws_access["Key"] = ""
aws_access["Secret"] = ""

In [44]:
#Download the model definition and weights from the S3 repository

conn = tinys3.Connection(aws_access["Key"], aws_access["Secret"], tls=True)

f = conn.get(bucket="juandoso-microtales", key='model_weights.h5')
f1 = open(local_path+'model_weights.h5','w')
f1.write(f.content)
f1.close()

f = conn.get(bucket="juandoso-microtales", key='model_definition.yaml')
f1 = open(local_path+'model_definition.yaml','w')
f1.write(f.content)
f1.close()

In [45]:
#Load the trained model
yaml_file = open(local_path+'model_definition.yaml', 'r').read()
model = model_from_yaml(yaml_file)

model.load_weights(local_path+'model_weights.h5')
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# the model will be used as a global constant
MODEL = model

In [46]:
# we limit ourselves to the following chars.
# Uppercase letters will be represented by prefixing them with a U
# - a trick proposed by Zygmunt Zajac http://fastml.com/one-weird-trick-for-training-char-rnns/
chars = u'\n !"#$%&\'()*+,-./0123456789:;<=>?@[\\]abcdefghijklmnopqrstuvwxyzU' + u'\xa0\xbb\xbf\xc1\xc9\xcd\xd1\xd3\xda\xe0\xe1\xe9\xed\xf1\xf3\xfa\xfc\u2015'
charset = set(chars)
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

def fix_char(c):
    if c.isupper():
        return 'U' + c.lower()
    elif c in charset:
        return c
    elif c in [u'\xa1', u'\xb0', u'\u201c', u'\u201d']:
        return '"'
    elif c in [u'\u2013', u'\u2014', u'\u2212', u'\u2500']:
        return '-'
    elif c == u'\u2026':
        return '...'
    else:
        return ''

def encode(text):
    return ''.join(fix_char(c) for c in text)

def decode(chars):
    upper = False
    for c in chars:
        if c == '#': continue
        if c == 'U':
            upper = True
        elif upper:
            upper = False
            yield c.upper()
        else:
            yield c

In [47]:
# helper function to sample an index from a probability array
def sample(a, temperature=1.0):
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    # np.random.multinomial throws an error if the probabilities
    # sum to > 1 - which they do due to finite precision
    while sum(a) > 1:
        a /= 1.000001
    return np.argmax(np.random.multinomial(1, a, 1))

# function to yield generated characters
def generate(model, seed, diversity):
    _, maxlen, _ = model.input_shape
    rseed = seed.rjust(maxlen)
    sentence = rseed[-maxlen:]
    while True:
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        yield next_char
        sentence = sentence[1:] + next_char

# generate n characters from seed
def generate_tale(model, seed, diversity, n):
    generator = decode(generate(model, seed, diversity))

    full_text = []
    for _ in range(n):
        next_char = generator.next()
        full_text.append(next_char)

    return ''.join(full_text)

In [48]:
TEMPERATURE = 0.75

# generate a new tale for a mention
def tale_to_tweet(mention):
    author = mention[0]
    seed = mention[1].replace("@talesmachine ", "")
    dseed = encode(seed)
    t = generate_tale(MODEL, dseed, TEMPERATURE, 139)
    return author, seed, t

# obtains the last mentions
def new_mentions(db_mentions):
    mentions = api.mentions_timeline()
    new_mentions = []
    for m in mentions:
        if (m.author.screen_name, m.text, m.id) in dic_mentions:
            continue
        else:
            db_mentions.append((m.author.screen_name, m.text, m.id))
            new_mentions.append((m.author.screen_name, m.text, m.id))
    
    return new_mentions

# create a list of responses for the new mentions
def new_tales(new_mentions):
    new_tales = []
    for m in new_mentions:
        new_tales.append(tale_to_tweet(m))
    return new_tales

# tweet the generated tales
def respond_with_mentions(new_tales):
    if new_tales:
        print("new tale:", new_tales)
        for t in new_tales:
            try:
                tweet = "Este cuento esta dedicado a @%s. %s ..." % (t[0], t[1])
                api.update_status(tweet)
                api.update_status(t[2])
            except Exception as e:
                print("Error at tweeting:", e)
                pass

In [49]:
# get the current mentions
mentions = api.mentions_timeline()

# database of current mentions
dic_mentions = []
for m in mentions:
    dic_mentions.append((m.author.screen_name, m.text, m.id))

In [50]:
# check mentions for control commands from my Twitter handle
# Commands will be preceded by the words "Heed my command!"
# for example: "Heed my command! temperature: 0.9"

def check_for_commands(mentions):
    for m in mentions:
        if (m[0] == "juandoso") & (m[1].find("Heed my command! ") > -1):
            mentions.remove(m)
            try:
                # Change the diversity factor
                if m[1].find("temperature:") > -1:
                    i = m[1].find("temperature:")
                    TEMPERATURE = float(m[1][i+13:])
                    message = "Changing diversity to {}".format(TEMPERATURE)
                    print(message)
                    api.update_status(message)
                    
            except Exception as e:
                print("Command failed", e)
                continue
            
    return mentions

In [51]:
#Refresh periodically
def check_and_tale():
    mentions = new_mentions(dic_mentions)
    mentions = check_for_commands(mentions)
    tales = new_tales(mentions)
    respond_with_mentions(tales)

In [52]:
# the service
import time, threading
def running_machine():
    #print("running...")
    check_and_tale()
    threading.Timer(66, running_machine).start()


In [ ]:
# start
print("starting microtales machine...")
running_machine()